In [4]:
import cvxpy as cp
import numpy as np
import time

In [5]:
import pandas as pd
dfs = pd.read_excel("/Users/yanjun/Dropbox/DRO_Conv_Regression(Zhengqing)/Concrete_data/Concrete_Data.xls");


In [12]:
dfs_train = dfs.head(400)

In [14]:
dfs_train.iloc[0,0]

540.0

In [15]:
dfs_train.columns

Index(['Cement (component 1)(kg in a m^3 mixture)',
       'Blast Furnace Slag (component 2)(kg in a m^3 mixture)',
       'Fly Ash (component 3)(kg in a m^3 mixture)',
       'Water  (component 4)(kg in a m^3 mixture)',
       'Superplasticizer (component 5)(kg in a m^3 mixture)',
       'Coarse Aggregate  (component 6)(kg in a m^3 mixture)',
       'Fine Aggregate (component 7)(kg in a m^3 mixture)', 'Age (day)',
       'Concrete compressive strength(MPa, megapascals) '],
      dtype='object')

In [24]:
dfs_np = dfs_train.values

In [102]:
dfs_train['Concrete compressive strength(MPa, megapascals) '];

In [25]:
X = dfs_np[:,0:7].T

In [26]:
X.shape

(7, 100)

In [27]:
Y = dfs_np[:,8].T

In [41]:
Y = Y.reshape((1,-1))

In [42]:
Y[0][0]

79.98611076

In [129]:
dfs_train = dfs.head(600)
dfs_np = dfs_train.values


X = dfs_np[:,0:7].T
Y = dfs_np[:,8].T
Y = Y.reshape((1,-1))
n = X.shape[1]
d = X.shape[0]

In [130]:
start_time = time.time()
        
Xi = cp.Variable((d,n))
g = cp.Variable(n)
C = cp.Variable(1)
h = cp.Variable(n)

cost = cp.sum(h)/n
constraints = [np.max(Xi) <= np.log(n)]
constraints = [-np.max(Xi) <= np.log(n)]
constraints += [np.max(Xi) <= C]
constraints += [-np.max(Xi) <= C]
#constraints += [h >= (Y-g)]
#constraints += [h >= (Y-g) for i in range(n)]
constraints += [h[i] >= (Y[0][i]-g[i]) for i in range(n)]
constraints += [h[i] >= -(Y[0][i]-g[i]) for i in range(n)]
constraints += [
#    g[i] - g[j] >= np.dot(Xi[:,j], X[:,i] - X[:,j]) for i in range(n) for j in range(n)
#     g[i] - g[j] >= np.sum([Xi[j*d+k]*(X[k,i] - X[k,j]) for k in range(d)])/d for i in range(n) for j in range(n)
    g[i] - g[j] >= Xi[:,j].T@(X[:,i] - X[:,j]) for i in range(n) for j in range(n)
]
prob = cp.Problem(cp.Minimize(cost),constraints)
prob.solve(cp.ECOS)
print(time.time() - start_time)

KeyboardInterrupt: 

In [43]:
Y.shape

(1, 100)

In [70]:
g.value[1]

61.88736576000021

In [59]:
Xi.shape

(7, 300)

In [73]:
def get_pred(x, Xi, X, g, n):
    mark = -100000;
    for j in range(n):
        tmp = g[j] + Xi[:,j].T@(x - X[:,j]);
        #print(tmp);
        if tmp > mark:
            mark = tmp;
    return mark      

In [76]:
get_pred(X[:,1], Xi.value, X, g.value, n)

61.88736576000021

In [ ]:
for iter in range(N):
    for j in range(k):
        n = (j+1)*50
        #X = np.random.randn(d, n)
        X = np.random.standard_t(10, size=(d,n))
        Y_star = np.zeros((1,n))
        for i in range(n):
            Y_star[0][i] = np.sqrt(sum(X[:,i]**2))
        Y = Y_star + 0.2* np.random.randn(1, n)
        
        start_time = time.time()
        
        Xi = cp.Variable((d,n))
        g = cp.Variable(n)
        C = cp.Variable(1)
        h = cp.Variable(n)

        cost = cp.sum(h)/n
        constraints = [np.max(Xi) <= np.log(n)]
        constraints = [-np.max(Xi) <= np.log(n)]
        constraints += [np.max(Xi) <= C]
        constraints += [-np.max(Xi) <= C]
        #constraints += [h >= (Y-g)]
        #constraints += [h >= (Y-g) for i in range(n)]
        constraints += [h[i] >= (Y[0][i]-g[i]) for i in range(n)]
        constraints += [h[i] >= -(Y[0][i]-g[i]) for i in range(n)]
        constraints += [
        #    g[i] - g[j] >= np.dot(Xi[:,j], X[:,i] - X[:,j]) for i in range(n) for j in range(n)
        #     g[i] - g[j] >= np.sum([Xi[j*d+k]*(X[k,i] - X[k,j]) for k in range(d)])/d for i in range(n) for j in range(n)
            g[i] - g[j] >= Xi[:,j].T@(X[:,i] - X[:,j]) for i in range(n) for j in range(n)
        ]
        prob = cp.Problem(cp.Minimize(cost),constraints)
        prob.solve(cp.ECOS)
        loss_l1 = np.mean(abs(g.value - Y_star))
        loss_l2 = np.sqrt(np.mean((g.value - Y_star)**2))
        re_loss1[iter][j] = loss_l1
        re_loss2[iter][j] = loss_l2
        print('j:',j+1,'/',k)
        print('iter:',iter+1,'/',N)
        print('l1_loss:',loss_l1)
        print('l2_loss:',loss_l2)
        print("--- %s seconds ---" % (time.time() - start_time))
        np.savetxt("l1-loss-l1.csv", re_loss1, delimiter=",")
        np.savetxt("l1-loss-l2.csv", re_loss2, delimiter=",")
        np.savetxt("l1-time.csv", re_time, delimiter=",")
    mean_loss1[0] = np.mean(re_loss1,0)*N / (iter+1)
    mean_loss2[0] = np.mean(re_loss2,0)*N / (iter+1)
    print('current l1 loss mean:', mean_loss1[0])
    print('current l2 loss mean:', mean_loss2[0])
    np.savetxt("l1-loss-l1-mean.csv", mean_loss1, delimiter=",")
    np.savetxt("l1-loss-l2-mean.csv", mean_loss2, delimiter=",")

In [5]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

In [99]:
X_linear= dfs_np[:,0:7]
Y_linear = dfs_np[:,8]

In [107]:
# 10-fold cross-validation with all three features
# instantiate model
lm = LinearRegression()

# store scores in scores object
# we can't use accuracy as our evaluation metric since that's only relevant for classification problems
# RMSE is not directly available so we will use MSE
scores = cross_val_score(lm, X_linear, Y_linear, cv=5, scoring='neg_mean_squared_error')
print(scores)

[-309.64355306 -209.67733558 -190.83155114 -156.66577193 -117.52471058]


In [108]:
lm.fit(X_linear, Y_linear)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [111]:
lm.predict(X_linear)

array([56.15775718, 56.28704215, 36.40034854, ..., 27.85329119,
       31.07256579, 35.09021813])

In [112]:
Y_linear

array([79.98611076, 61.88736576, 40.26953526, ..., 23.69660064,
       32.76803638, 32.40123514])

In [92]:
X_linear.shape

(300, 7)

In [93]:
Y_linear.shape

(300, 1)

In [131]:
from sklearn.model_selection import KFold # import KFold
kf = KFold(n_splits=5) # Define the split - into 2 folds 
#kf.get_n_splits(X_linear) # returns the number of splitting iterations in the cross-validator
#print(kf) 
#KFold(n_splits=2, random_state=None, shuffle=True)

for train_index, test_index in kf.split(X_linear):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X_linear[train_index], X_linear[test_index]
    y_train, y_test = Y_linear[train_index], Y_linear[test_index]
    lm = LinearRegression()
    lm.fit(X_train, y_train)
    y_pred = lm.predict(X_test)
    print(np.sqrt(np.sum((y_test - y_pred)**2)/y_test.shape[0]))


    

17.596691537306985
14.480239486444642
13.814179351013143
12.516619828391102
10.840881448322095


In [135]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_linear, Y_linear, test_size=0.7)

In [134]:
X_train.shape

(309, 7)

In [121]:
X_linear.shape

(1030, 7)

In [145]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_linear, Y_linear, test_size=0.9, random_state = 1)

In [146]:
lm = LinearRegression()
lm.fit(X_train, y_train)
y_pred = lm.predict(X_test)
print(np.sqrt(np.sum((y_test - y_pred)**2)/y_test.shape[0]))

12.964949693073052


In [147]:
y_train.reshape(-1,1).T[0][2]

33.947729812

In [148]:
X = X_train.T
Y = y_train.reshape(-1,1).T

n = X.shape[1]
d = X.shape[0]
start_time = time.time()
        
Xi = cp.Variable((d,n))
g = cp.Variable(n)
C = cp.Variable(1)
h = cp.Variable(n)

cost = cp.sum(h)/n
constraints = [np.max(Xi) <= np.log(n)]
constraints = [-np.max(Xi) <= np.log(n)]
constraints += [np.max(Xi) <= C]
constraints += [-np.max(Xi) <= C]
#constraints += [h >= (Y-g)]
#constraints += [h >= (Y-g) for i in range(n)]
constraints += [h[i] >= (Y[0][i]-g[i]) for i in range(n)]
constraints += [h[i] >= -(Y[0][i]-g[i]) for i in range(n)]
constraints += [
#    g[i] - g[j] >= np.dot(Xi[:,j], X[:,i] - X[:,j]) for i in range(n) for j in range(n)
#     g[i] - g[j] >= np.sum([Xi[j*d+k]*(X[k,i] - X[k,j]) for k in range(d)])/d for i in range(n) for j in range(n)
    g[i] - g[j] >= Xi[:,j].T@(X[:,i] - X[:,j]) for i in range(n) for j in range(n)
]
prob = cp.Problem(cp.Minimize(cost),constraints)
prob.solve(cp.ECOS)
print(time.time() - start_time)

34.84214210510254


In [151]:
def get_pred(x, Xi, X, g, n):
    mark = -100000;
    for j in range(n):
        tmp = g[j] + Xi[:,j].T@(x - X[:,j]);
        #print(tmp);
        if tmp >= mark:
            mark = tmp;
    return mark 

MSE_convex = 0
for i in range(X_test.shape[0]):
    y_tmp = get_pred(X_test.T[:,i], Xi.value, X, g.value, n)
    MSE_convex += (y_tmp - y_test[i])**2
MSE_convex = MSE_convex/X_test.shape[0]
print(np.sqrt(MSE_convex))

In [158]:
MSE_convex = 0
for i in range(X_test.shape[0]):
    y_tmp = get_pred(X_test.T[:,i], Xi.value, X, g.value, n)
    MSE_convex += (y_tmp - y_test[i])**2
MSE_convex = MSE_convex/X_test.shape[0]
print(np.sqrt(MSE_convex))

75.56505649806378


In [157]:
MSE_convex

5710.077763555571

In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
import time
import cvxpy as cp



dfs = pd.read_excel("/Users/yanjun/Dropbox/DRO_Conv_Regression(Zhengqing)/Concrete_data/Concrete_Data.xls");

normalized_df=(dfs-dfs.mean())/dfs.std()

dfs_np = normalized_df.values

X_linear= dfs_np[:,0:7]

# Try concave this time
Y_linear = -dfs_np[:,8]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_linear, Y_linear, test_size=0.7, random_state = 1)

lm = LinearRegression()
lm.fit(X_train, y_train)
y_pred = lm.predict(X_test)
print("This is the sqrt MSE for linear regression:", np.sqrt(np.sum((y_test - y_pred)**2)/y_test.shape[0]))


X = X_train.T
Y = y_train.reshape(-1,1).T

n = X.shape[1]
d = X.shape[0]
start_time = time.time()
print("training size is:", n)
        
Xi = cp.Variable((d,n))
g = cp.Variable(n)
C = cp.Variable(1)
h = cp.Variable(n)

cost = cp.sum(h)/n
constraints = [np.max(Xi) <= np.log(n)]
constraints = [-np.max(Xi) <= np.log(n)]
constraints += [np.max(Xi) <= C]
constraints += [-np.max(Xi) <= C]
#constraints += [h >= (Y-g)]
#constraints += [h >= (Y-g) for i in range(n)]
constraints += [h[i] >= (Y[0][i]-g[i]) for i in range(n)]
constraints += [h[i] >= -(Y[0][i]-g[i]) for i in range(n)]
constraints += [
#    g[i] - g[j] >= np.dot(Xi[:,j], X[:,i] - X[:,j]) for i in range(n) for j in range(n)
#     g[i] - g[j] >= np.sum([Xi[j*d+k]*(X[k,i] - X[k,j]) for k in range(d)])/d for i in range(n) for j in range(n)
    g[i] - g[j] >= Xi[:,j].T@(X[:,i] - X[:,j]) for i in range(n) for j in range(n)
]
prob = cp.Problem(cp.Minimize(cost),constraints)
prob.solve(cp.ECOS)
print("CVS running time is:", time.time() - start_time)

def get_pred(x, Xi, X, g, n):
    mark = -100000;
    for j in range(n):
        tmp = g[j] + Xi[:,j].T@(x - X[:,j]);
        #print(tmp);
        if tmp >= mark:
            mark = tmp;
    return mark 

MSE_convex = 0
for i in range(X_test.shape[0]):
    y_tmp = get_pred(X_test.T[:,i], Xi.value, X, g.value, n)
    MSE_convex += (y_tmp - y_test[i])**2
MSE_convex = MSE_convex/X_test.shape[0]
print("This is the test sqrt MSE for CVS:", np.sqrt(MSE_convex))

MSE_convex_train = 0
for i in range(X_train.shape[0]):
    y_tmp = get_pred(X_train.T[:,i], Xi.value, X, g.value, n)
    MSE_convex_train += (y_tmp - y_train[i])**2
MSE_convex_train = MSE_convex_train/X_train.shape[0]
print("This is the train sqrt MSE for CVS:", np.sqrt(MSE_convex_train))

KeyboardInterrupt: 

In [168]:
dfs_weather = pd.read_csv("/Users/yanjun/Dropbox/DRO_Conv_Regression(Zhengqing)/weather_data(backup)/DLY_2019caQ1.csv");
dfs_weather_1 = dfs_weather.drop("SLOAD (1000 lbs)", 1)
dfs_weather_2 = dfs_weather_1.dropna()
dfs_weather_X_train = dfs_weather_2[['NOX_RATE (lbs/mmBtu)', 'CO2_MASS (tons)', 'SO2_MASS (tons)', 'NOX_MASS (tons)']]
dfs_weather_y_train = dfs_weather_2['HEAT_INPUT (mmBtu)']
dfs_weather_X_train_log = np.log(dfs_weather_X_train)
dfs_pre = pd.concat([dfs_weather_X_train_log, dfs_weather_y_train],axis=1)
dfs_pre2 = dfs_pre.replace([np.inf, -np.inf], np.nan).dropna()

In [180]:
dfs_weather_1 = dfs_weather.drop("SLOAD (1000 lbs)", 1)

In [181]:
dfs_weather_1.shape

(21960, 17)

In [182]:
dfs_weather_2 = dfs_weather_1.dropna()

In [183]:
dfs_weather_2.shape

(8831, 17)

In [193]:
dfs_weather_X_train = dfs_weather_2[['NOX_RATE (lbs/mmBtu)', 'CO2_MASS (tons)', 'SO2_MASS (tons)', 'NOX_MASS (tons)']]

In [194]:
dfs_weather_y_train = dfs_weather_2['HEAT_INPUT (mmBtu)']

In [195]:
dfs_weather_X_train_log = np.log(dfs_weather_X_train)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


In [199]:
dfs_weather_X_train_log_nonan = dfs_weather_X_train_log.dropna()

In [205]:
normalized_df=(dfs_weather_X_train_log-dfs_weather_X_train_log.mean())/dfs_weather_X_train_log.std()

In [210]:
dfs_weather_X_train_log.isnull().values.any()


False

In [214]:
dfs_weather_X_train_log.values.mean()

-inf

In [216]:
dfs_weather_X_train_log.shape

(8831, 4)

In [218]:
dfs_weather_X_train_log.min()

NOX_RATE (lbs/mmBtu)        -inf
CO2_MASS (tons)        -4.017384
SO2_MASS (tons)             -inf
NOX_MASS (tons)             -inf
dtype: float64

In [211]:
dfs_weather_X_train_log.std()

NOX_RATE (lbs/mmBtu)         NaN
CO2_MASS (tons)         1.730486
SO2_MASS (tons)              NaN
NOX_MASS (tons)              NaN
dtype: float64

In [219]:
dfs_weather_X_train_log2 = dfs_weather_X_train_log.replace([np.inf, -np.inf], np.nan).dropna()

In [223]:
normalized_df=(dfs_weather_X_train_log2-dfs_weather_X_train_log2.mean())/dfs_weather_X_train_log2.std()

In [224]:
dfs_weather_X_train_log2.std()

NOX_RATE (lbs/mmBtu)    0.751494
CO2_MASS (tons)         1.053262
SO2_MASS (tons)         0.978659
NOX_MASS (tons)         0.915488
dtype: float64

In [236]:
dfs_pre = pd.concat([dfs_weather_X_train_log, dfs_weather_y_train],axis=1)

In [237]:
dfs_pre.shape

(8831, 5)

In [239]:
dfs_pre2 = dfs_pre.replace([np.inf, -np.inf], np.nan).dropna()

In [240]:
dfs_pre2.shape

(6342, 5)

In [11]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

In [12]:
dfs_weather = pd.read_csv("/Users/yanjun/Dropbox/DRO_Conv_Regression(Zhengqing)/weather_data(backup)/DLY_2019caQ1.csv");
dfs_weather_1 = dfs_weather.drop("SLOAD (1000 lbs)", 1)
dfs_weather_2 = dfs_weather_1.dropna()
dfs_weather_X_train = dfs_weather_2[['NOX_RATE (lbs/mmBtu)', 'CO2_MASS (tons)', 'SO2_MASS (tons)', 'NOX_MASS (tons)']]
dfs_weather_y_train = dfs_weather_2['HEAT_INPUT (mmBtu)']
dfs_weather_X_train_log = np.log(dfs_weather_X_train)
dfs_pre = pd.concat([dfs_weather_X_train_log, dfs_weather_y_train],axis=1)
dfs_pre2 = dfs_pre.replace([np.inf, -np.inf], np.nan).dropna()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


In [21]:
kernel_width = 0.1
test_ratio = 0.5

K = 100
result = np.zeros((K,8))

import pandas as pd

normalized_df=(dfs_pre2-dfs_pre2.mean())/dfs_pre2.std()

dfs_np = normalized_df.head(600).values

X_linear= dfs_np[:,0:3]
Y_linear = -dfs_np[:,-1]



for j in range(K):
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X_linear, Y_linear, test_size=test_ratio, random_state = j)

    lm = LinearRegression()
    lm.fit(X_train, y_train)
    y_pred = lm.predict(X_test)
    #print("This is the sqrt MSE for linear regression:", np.sqrt(np.sum((y_test - y_pred)**2)/y_test.shape[0]))
    result[j][1] =  np.sum((y_test - y_pred)**2)/y_test.shape[0]
    
    y_pred_train = lm.predict(X_train)
    result[j][0] =  np.sum((y_train - y_pred_train)**2)/y_train.shape[0]


    X = X_train.T
    Y = y_train.reshape(-1,1).T

    n = X.shape[1]
    d = X.shape[0]
    start_time = time.time()
    print("training size is:", n)

    Xi = cp.Variable((d,n))
    g = cp.Variable(n)
    C = cp.Variable(1)
    h = cp.Variable(n)

    cost = cp.sum(h)/n
    constraints = [np.max(Xi) <= np.log(n)]
    constraints = [-np.max(Xi) <= np.log(n)]
    constraints += [np.max(Xi) <= C]
    constraints += [-np.max(Xi) <= C]
    #constraints += [h >= (Y-g)]
    #constraints += [h >= (Y-g) for i in range(n)]
    constraints += [h[i] >= (Y[0][i]-g[i]) for i in range(n)]
    constraints += [h[i] >= -(Y[0][i]-g[i]) for i in range(n)]
    constraints += [
    #    g[i] - g[j] >= np.dot(Xi[:,j], X[:,i] - X[:,j]) for i in range(n) for j in range(n)
    #     g[i] - g[j] >= np.sum([Xi[j*d+k]*(X[k,i] - X[k,j]) for k in range(d)])/d for i in range(n) for j in range(n)
        g[i] - g[j] >= Xi[:,j].T@(X[:,i] - X[:,j]) for i in range(n) for j in range(n)
    ]
    prob = cp.Problem(cp.Minimize(cost),constraints)

    print("Start CSV training")
    prob.solve(cp.ECOS)
    print("Finished CVS training. CVS running time is:", time.time() - start_time)

    def get_pred(x, Xi, X, g, n):
        mark = -100000;
        for j in range(n):
            tmp = g[j] + Xi[:,j].T@(x - X[:,j]);
            #print(tmp);
            if tmp >= mark:
                mark = tmp;
        return mark 

    MSE_convex = 0
    for i in range(X_test.shape[0]):
        y_tmp = get_pred(X_test.T[:,i], Xi.value, X, g.value, n)
        MSE_convex += (y_tmp - y_test[i])**2
    MSE_convex = MSE_convex/X_test.shape[0]
    #print("This is the sqrt MSE for CVS:", np.sqrt(MSE_convex))
    result[j][3] = MSE_convex
    
    MSE_convex = 0
    for i in range(X_train.shape[0]):
        y_tmp = get_pred(X_train.T[:,i], Xi.value, X, g.value, n)
        MSE_convex += (y_tmp - y_train[i])**2
    MSE_convex = MSE_convex/X_train.shape[0]
    
    result[j][2] = MSE_convex
    
    
    ### l2 convex
    Xi = cp.Variable((d,n))
    g = cp.Variable(n)
    C = cp.Variable(1)
    h = cp.Variable(n)

    cost = cp.sum(h**2)/n
    constraints = [np.max(Xi) <= 20]
    constraints = [-np.max(Xi) <= 20]
    constraints += [np.max(Xi) <= C]
    constraints += [-np.max(Xi) <= C]
    #constraints += [h >= (Y-g)]
    #constraints += [h >= (Y-g) for i in range(n)]
    constraints += [h[i] >= (Y[0][i]-g[i]) for i in range(n)]
    constraints += [h[i] >= -(Y[0][i]-g[i]) for i in range(n)]
    constraints += [
    #    g[i] - g[j] >= np.dot(Xi[:,j], X[:,i] - X[:,j]) for i in range(n) for j in range(n)
    #     g[i] - g[j] >= np.sum([Xi[j*d+k]*(X[k,i] - X[k,j]) for k in range(d)])/d for i in range(n) for j in range(n)
        g[i] - g[j] >= Xi[:,j].T@(X[:,i] - X[:,j]) for i in range(n) for j in range(n)
    ]
    prob = cp.Problem(cp.Minimize(cost),constraints)

    print("Start L2 CSV training")
    prob.solve()
    print("Finished L2 CVS training. CVS running time is:", time.time() - start_time)
    
    
    MSE_convex = 0
    for i in range(X_test.shape[0]):
        y_tmp = get_pred(X_test.T[:,i], Xi.value, X, g.value, n)
        MSE_convex += (y_tmp - y_test[i])**2
    MSE_convex = MSE_convex/X_test.shape[0]
    #print("This is the sqrt MSE for CVS:", np.sqrt(MSE_convex))
    result[j][5] = MSE_convex
    
    MSE_convex = 0
    for i in range(X_train.shape[0]):
        y_tmp = get_pred(X_train.T[:,i], Xi.value, X, g.value, n)
        MSE_convex += (y_tmp - y_train[i])**2
    MSE_convex = MSE_convex/X_train.shape[0]
    
    result[j][4] = MSE_convex
    
    
    print("Start kernel fitting")
    y_pred_kernel = predict(X_test, X_train, y_train, kernel_width, kernel_func=gaussian_kernel)
    #print("End kernel fitting, This is the sqrt MSE:", np.sqrt(np.sum((y_test - y_pred_kernel)**2)/y_test.shape[0]))
    result[j][7] = np.sum((y_test - y_pred_kernel)**2)/y_test.shape[0];
    
    y_pred_kernel = predict(X_train, X_train, y_train, kernel_width, kernel_func=gaussian_kernel)
    result[j][6] = np.sum((y_train - y_pred_kernel)**2)/y_train.shape[0];
    
    print("statistics:", result[j])
    np.savetxt("/Users/yanjun/Dropbox/DRO_Conv_Regression(Zhengqing)/weather_data(backup)/Convex_regression_weatherDate_local_0728.csv", result, delimiter=",")

    
    

training size is: 300
Start CSV training


SolverError: Solver 'ECOS' failed. Try another solver, or solve with verbose=True for more information.

In [14]:
def gaussian_kernel(x, x0, h):
    return np.asarray([np.exp(- 0.5 * np.sum(np.power((z - x0) / h, 2)) ) for z in x])
    #return np.exp(- 0.5 * np.power((x - x0) / h, 2) )

def Kernel_predict(x_test, x_train, y_train, bandwidth, kernel_func=gaussian_kernel):
    return np.array([(kernel_func(x_train, x0, bandwidth).dot(y_train) ) / 
                     kernel_func(x_train, x0, bandwidth).sum() for x0 in x_test])

In [35]:
y_pred_kernel = predict(X_test, X_train, y_train, 0.1, kernel_func=gaussian_kernel)

In [29]:
gaussian_kernel(X_train, X_train[0], 0.5).shape

(309,)

In [30]:
y_train.shape

(309,)

In [31]:
gaussian_kernel(X_train, X_train[0], 0.5).dot(y_train)

0.6618578378176662

In [32]:
X_test.shape

(721, 7)

In [51]:
pd

<module 'pandas' from '/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/__init__.py'>

In [54]:
dfs_wage = pd.read_csv("/Users/yanjun/Dropbox/DRO_Conv_Regression(Zhengqing)/wage_data/wage_transformed.csv");
dfs_wage_data = dfs_wage(['Wage', 'Education', 'Experience'])

TypeError: 'DataFrame' object is not callable

In [67]:
dfs_wage_data = dfs_wage[['Wage', 'Education', 'Experience']].head(600)

In [68]:
dfs_wage_data.head(10)

,Wage,Education,Experience
0,354.94,3.583181,45
1,370.37,5.159780,9
2,754.94,7.430084,46
3,593.54,8.916100,36
4,377.23,18.488426,22
5,284.90,4.299817,51
6,561.13,8.916100,34
7,264.06,8.916100,0
8,1643.83,12.839185,18
9,474.83,8.916100,17


In [ ]:
kernel_width = 0.1
test_ratio = 0.4


K = 10
result = np.zeros((K,6))

import pandas as pd

normalized_df=(dfs_wage_data-dfs_wage_data.mean())/dfs_wage_data.std()

dfs_np = normalized_df.values

X_linear= dfs_np[:,1:3]
Y_linear = -dfs_np[:,0]



for j in range(10):
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X_linear, Y_linear, test_size=test_ratio, random_state = j)

    lm = LinearRegression()
    lm.fit(X_train, y_train)
    y_pred = lm.predict(X_test)
    #print("This is the sqrt MSE for linear regression:", np.sqrt(np.sum((y_test - y_pred)**2)/y_test.shape[0]))
    result[j][1] =  np.sum((y_test - y_pred)**2)/y_test.shape[0]
    
    y_pred_train = lm.predict(X_train)
    result[j][0] =  np.sum((y_train - y_pred_train)**2)/y_train.shape[0]

    print("Linear regression training, test MSE:", result[j][0], ' ', result[j][1])
    
    X = X_train.T
    Y = y_train.reshape(-1,1).T

    n = X.shape[1]
    d = X.shape[0]
    start_time = time.time()
    print("training size is:", n)

    Xi = cp.Variable((d,n))
    g = cp.Variable(n)
    C = cp.Variable(1)
    h = cp.Variable(n)

    cost = cp.sum(h)/n
    constraints = [np.max(Xi) <= np.log(n)]
    constraints = [-np.max(Xi) <= np.log(n)]
    constraints += [np.max(Xi) <= C]
    constraints += [-np.max(Xi) <= C]
    #constraints += [h >= (Y-g)]
    #constraints += [h >= (Y-g) for i in range(n)]
    constraints += [h[i] >= (Y[0][i]-g[i]) for i in range(n)]
    constraints += [h[i] >= -(Y[0][i]-g[i]) for i in range(n)]
    constraints += [
    #    g[i] - g[j] >= np.dot(Xi[:,j], X[:,i] - X[:,j]) for i in range(n) for j in range(n)
    #     g[i] - g[j] >= np.sum([Xi[j*d+k]*(X[k,i] - X[k,j]) for k in range(d)])/d for i in range(n) for j in range(n)
        g[i] - g[j] >= Xi[:,j].T@(X[:,i] - X[:,j]) for i in range(n) for j in range(n)
    ]
    prob = cp.Problem(cp.Minimize(cost),constraints)

    print("Start CSV training")
    prob.solve()
    print("Finished CVS training. CVS running time is:", time.time() - start_time)
    
    

    def get_pred(x, Xi, X, g, n):
        mark = -100000;
        for j in range(n):
            tmp = g[j] + Xi[:,j].T@(x - X[:,j]);
            #print(tmp);
            if tmp >= mark:
                mark = tmp;
        return mark 

    MSE_convex = 0
    for i in range(X_test.shape[0]):
        y_tmp = get_pred(X_test.T[:,i], Xi.value, X, g.value, n)
        MSE_convex += (y_tmp - y_test[i])**2
    MSE_convex = MSE_convex/X_test.shape[0]
    #print("This is the sqrt MSE for CVS:", np.sqrt(MSE_convex))
    result[j][3] = MSE_convex
    
    MSE_convex = 0
    for i in range(X_train.shape[0]):
        y_tmp = get_pred(X_train.T[:,i], Xi.value, X, g.value, n)
        MSE_convex += (y_tmp - y_train[i])**2
    MSE_convex = MSE_convex/X_train.shape[0]
    
    result[j][2] = MSE_convex
    
    print("CVS training, test MSE:", result[j][2], ' ', result[j][3])
    
    print("Start kernel fitting")
    y_pred_kernel = predict(X_test, X_train, y_train, kernel_width, kernel_func=gaussian_kernel)
    #print("End kernel fitting, This is the sqrt MSE:", np.sqrt(np.sum((y_test - y_pred_kernel)**2)/y_test.shape[0]))
    result[j][5] = np.sum((y_test - y_pred_kernel)**2)/y_test.shape[0];
    
    y_pred_kernel = predict(X_train, X_train, y_train, kernel_width, kernel_func=gaussian_kernel)
    result[j][4] = np.sum((y_train - y_pred_kernel)**2)/y_train.shape[0];
    
    print("Kernel regression training, test MSE:", result[j][4], ' ', result[j][5])
    print("statistics:", result[j])
    np.savetxt("/Users/yanjun/Dropbox/DRO_Conv_Regression(Zhengqing)/wage_data/Convex_regression_wagedata_0727.csv", result, delimiter=",")

    
    

Linear regression training, test MSE: 0.9442779094228139   0.7158387730652573
training size is: 360
Start CSV training


In [73]:
np.abs(-1)

1